In [11]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/250.9 kB 787.7 kB/s eta 0:00:01
   ------------------------------------- -- 235.5/250.9 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd
from scipy.stats import f
from scipy.spatial.distance import mahalanobis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Load data
df = pd.read_excel("data.xlsx",sheet_name="2022",index_col = "Наименование")

In [14]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [15]:
df["Доходы консолидированных бюджетов субъектов Российской Федерации (млн)"].replace(
                                regex={',': '.'," ":"",'–':0},inplace=True)

C:\Users\Максим\AppData\Local\Temp\ipykernel_10364\3488075491.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Доходы консолидированных бюджетов субъектов Российской Федерации (млн)"].replace(


In [16]:
df_filtered = pd.DataFrame({"X1":df["Число дорожно-транспортных происшествий и пострадавших в них на 100 000 человек населения"],
                            "X2":df["Смертность населения старше трудоспособного возраста, на 100 000 человек населения соответствующего возраста"],
                           "X3":df["Продажа сильно алкогольной продукции населению(тысяч декалитров)"].div(df["Численность населения, тыс. человек"]),
                            "X4":df["Стоимость минимального (условного) набора потребительских товаров и услуг"].astype(float),
                            "X5":df["Число спортивных сооружений"].div(df["Численность населения, тыс. человек"]),
                            "X6":df["Доходы консолидированных бюджетов субъектов Российской Федерации (млн)"].astype(float).div(df["Численность населения, тыс. человек"]),
                            "X7":df["Предварительно расследовано преступлений, совершенных в состоянии алкогольного опьянения"].div(df["Численность населения, тыс. человек"]),
                            "X8":df["Среднедушевые доходы населения (в месяц), руб."],
                            "X9":df["Численность студентов, обучающихся по программам бакалавриата, специалитета, магистратуры на 10 000 человек населения, всего"].astype(float)
                           })

In [17]:
df_filtered

,X1,X2,X3,X4,X5,X6,X7,X8,X9
Наименование,,,,,,,,,
Алтайский край,107.2,610.1,0.549898,112.0,2.336509,0.004867,3.361268,26010,243.0
Амурская область,150.2,769.2,1.013971,109.5,2.491170,0.131129,3.414995,39626,194.0
Архангельская область без автономного округа,76.1,629.2,1.325179,108.4,1.947488,0.078067,3.030572,37810,180.0
Астраханская область,97.9,489.8,0.543408,109.0,1.411707,0.043660,1.775376,26833,294.0
Белгородская область,68.8,487.3,0.521666,108.6,2.850876,0.033237,1.204854,35612,311.0
...,...,...,...,...,...,...,...,...,...
Чеченская Республика,12.9,181.4,0.011156,110.3,1.221204,0.028307,0.102369,26397,231.0
Чувашская Республика — Чувашия,83.8,588.7,0.935477,110.0,2.816189,0.031015,1.914228,23619,299.0
Чукотский автономный округ,35.5,820.0,1.301360,110.6,1.566645,7.996818,5.075930,99905,20.0


In [18]:
df_filtered.dtypes

X1    float64
X2    float64
X3    float64
X4    float64
X5    float64
X6    float64
X7    float64
X8      int64
X9    float64
dtype: object

In [19]:
df_filtered["X6"] - df_filtered["X6"].mean(axis=0)/df_filtered["X6"]

Наименование
Алтайский край                                 -60.516068
Амурская область                                -2.115225
Архангельская область без автономного округа    -3.695124
Астраханская область                            -6.703025
Белгородская область                            -8.829254
                                                  ...    
Чеченская Республика                           -10.377874
Чувашская Республика — Чувашия                  -9.466314
Чукотский автономный округ                       7.959983
Ямало-Ненецкий автономный округ                 -0.024526
Ярославская область                             -6.715452
Name: X6, Length: 85, dtype: float64

In [21]:
sdf = pd.DataFrame()
for col in df_filtered.columns:
    sdf[col] = (df_filtered[col] - df_filtered[col].mean(axis=0))/df_filtered[col].std()

In [22]:
sdf["Наименование"] = df_filtered.index

In [23]:
sdf.set_index("Наименование",inplace=True)

In [24]:
sdf

,X1,X2,X3,X4,X5,X6,X7,X8,X9
Наименование,,,,,,,,,
Алтайский край,0.423721,0.439726,-0.646954,1.676521,0.920458,-0.216255,0.922800,-0.626353,0.089137
Амурская область,1.932733,1.687317,0.722511,0.050502,1.192731,-0.122002,0.966786,0.206284,-0.368074
Архангельская область без автономного округа,-0.667681,0.589500,1.640880,-0.664946,0.235605,-0.161612,0.652055,0.095233,-0.498706
Астраханская область,0.097353,-0.503612,-0.666108,-0.274702,-0.707612,-0.187296,-0.375588,-0.576026,0.565010
Белгородская область,-0.923862,-0.523216,-0.730266,-0.534865,1.825977,-0.195077,-0.842681,-0.039178,0.723634
...,...,...,...,...,...,...,...,...,...
Чеченская Республика,-2.885578,-2.921945,-2.236771,0.570828,-1.042984,-0.198758,-1.745298,-0.602688,-0.022833
Чувашская Республика — Чувашия,-0.397463,0.271917,0.490879,0.375706,1.764913,-0.196736,-0.261909,-0.772566,0.611664
Чукотский автономный округ,-2.092469,2.085667,1.570590,0.765950,-0.434851,5.749655,2.326612,3.892426,-1.991640


In [25]:
sdf["X6"] = [-0.832256599,
0.517564476,
0.155036569,
-0.359749214,
-0.222913948,
-0.403474737,
-0.406735079,
-0.492329036,
-0.282278512,
-0.386556673,
0.73188267,
0.434812446,
1.34589173,
-0.388519731,
-0.653255706,
-0.543854591,
-0.291236648,
-0.267983437,
2.0575036,
-0.322220544,
0.041703624,
-0.321621748,
-0.316242875,
-0.420316319,
-0.525390417,
-0.326985368,
-0.324499661,
-0.277539357,
-0.373201267,
2.0997129,
-0.188240646,
0.293967434,
4.95772489,
-0.309071346,
-0.260027907,
-0.339015896,
-0.0558568362,
-0.34632205,
-0.326353313,
-0.394927281,
-0.270026224,
-0.191349951,
-0.280892029,
-0.490077504,
0.140997861,
-0.419513275,
0.232711785,
-0.583393156,
-0.392435482,
-0.408194646,
-0.117387142,
0.298226211,
-0.990065046,
-0.430188766,
-0.27650776,
-0.17593093,
-0.450680697,
-0.202190013,
1.34630413,
-0.393110994,
-0.427326287,
-0.34977747,
-0.249657056,
0.131735835,
-0.491780111,
-0.990065046,
-0.2559698,
-0.990065046,
-0.356003993,
-0.509813052,
-0.305180798,
-0.275965286,
0.87357587,
-0.345618213,
0.211570693,
-0.351707674,
-0.460731175,
0.263057386,
0.713163046,
-0.40467678,
-0.337396802,
-0.43700844,
4.80229078,
3.11894892,
-0.198719471]

In [26]:
sdf

,X1,X2,X3,X4,X5,X6,X7,X8,X9
Наименование,,,,,,,,,
Алтайский край,0.423721,0.439726,-0.646954,1.676521,0.920458,-0.832257,0.922800,-0.626353,0.089137
Амурская область,1.932733,1.687317,0.722511,0.050502,1.192731,0.517564,0.966786,0.206284,-0.368074
Архангельская область без автономного округа,-0.667681,0.589500,1.640880,-0.664946,0.235605,0.155037,0.652055,0.095233,-0.498706
Астраханская область,0.097353,-0.503612,-0.666108,-0.274702,-0.707612,-0.359749,-0.375588,-0.576026,0.565010
Белгородская область,-0.923862,-0.523216,-0.730266,-0.534865,1.825977,-0.222914,-0.842681,-0.039178,0.723634
...,...,...,...,...,...,...,...,...,...
Чеченская Республика,-2.885578,-2.921945,-2.236771,0.570828,-1.042984,-0.337397,-1.745298,-0.602688,-0.022833
Чувашская Республика — Чувашия,-0.397463,0.271917,0.490879,0.375706,1.764913,-0.437008,-0.261909,-0.772566,0.611664
Чукотский автономный округ,-2.092469,2.085667,1.570590,0.765950,-0.434851,4.802291,2.326612,3.892426,-1.991640


In [27]:
train_dict = dict()

In [33]:
def gen_train(data, train):
    train_df = pd.DataFrame()
    for cl, region in train.items():
        ts = data[["X1","X2","X3","X4","X5","X6","X7","X8""X9"]].loc[region]
        ts["Y"] = cl
        pd.concat([train_df,ts])
    return train_df
        

In [29]:
# Проведем расчет матрицы рассеяния.
def scatter_matrix(samples):
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    d = samples - samples.mean()
    res = np.zeros((d.shape[1], d.shape[1]))
    for _, row in d.iterrows():
        col = row.to_frame()
        res += col @ col.T
    return res

# Проведем расчет внутриклассовых матриц разброса.

def classes_scatter_matrix(samples, labels):
    A = np.zeros((samples.shape[1], samples.shape[1]))
    for cls in labels.unique():
        A += scatter_matrix(samples[labels == cls])
    return A


In [30]:
cov = pd.DataFrame(
    classes_scatter_matrix(train_data[["X1","X2","X3","X4","X5","X6","X7","X8""X9"]], train_data.Class) / (train_data.shape[0] - train_data.Class.unique().size), 
    index=["X1","X2","X3","X4","X5","X6","X7","X8""X9"], 
    columns=["X1","X2","X3","X4","X5","X6","X7","X8""X9"]
)


NameError: name 'train_data' is not defined

In [31]:
lda = LinearDiscriminantAnalysis().fit(train_data[["X1","X2","X3","X4","X5","X6","X7","X8""X9"]], train_data["Y"])

#Рассчитаем оценки математических ожиданий в классах
means = pd.DataFrame(lda.means_, index=lda.classes_, columns=["X1","X2","X3","X4","X5","X6","X7","X8""X9"])
means


NameError: name 'train_data' is not defined

In [32]:
# Определим линейные дискриминантные функции Фишера
def get_df_coef(lda, features):
    if lda.classes_.size == 2: #если 2 класса, то lda вернет разность коэффициентов функций
        return pd.DataFrame(
            np.concatenate([lda.intercept_, lda.coef_.ravel()]),
            index=["Intercept"] + features,
            columns=["Discriminant F"]
        )
# если 3 и более классов, то lda вернет несколько функций (аналогично ППП Statistica)
    return pd.DataFrame(
        np.vstack([lda.intercept_, lda.coef_.T]),
        index=["Intercept"] + features,
        columns=lda.classes_
    )

In [34]:
df_coef = get_df_coef(lda, FEATURES) # применим эту функцию для наших параметров

df_coef

NameError: name 'lda' is not defined

In [35]:
# Зададим функцию расчета квадратов расстояний Махаланобиса
def calc_mahalanobis_square_dist(centers, samples, cov):
    res = pd.DataFrame(index=samples.index, columns=centers.index) # создаем матрицу расстояний между классами и объекnами
    for i in centers.index: # проходим по всем центрам классов
        for j in samples.index: # проходим по всем объектам
            # считаем расстояние между текущим клас-сом и объектом
            res[i][j] = mahalanobis(centers.loc[i], samples.loc[j], np.linalg.inv(cov)) ** 2 
    return res


In [36]:
center_distance = calc_mahalanobis_square_dist(means, means, cov)
center_distance

NameError: name 'means' is not defined

In [37]:
# Выведем оценки априорных вероятностей
lda.priors_

NameError: name 'lda' is not defined

In [38]:
# определим функцию вывода номеров классов для переданных X.
def LDA_predict(lda, X):
    return pd.DataFrame(
        lda.predict(X),
        columns=["Class"],
        index=X.index
    )


In [39]:
lda_predict_train = LDA_predict(lda, train_data[FEATURES])
# сравниваем полученные классы с теми, которые были заданы в начале, и смотрим долю совпадения
(lda_predict_train.Class == train_data.Class).value_counts(normalize=True)


NameError: name 'lda' is not defined

In [40]:
# Получим полную картину классификации всех объектов. 

lda_predict = LDA_predict(lda, data[FEATURES])
lda_predict.head()

data_to_excel["Results lda"] = lda_predict

NameError: name 'lda' is not defined

In [41]:
# Выведем квадраты расстояний Махаланобиса между класса-ми и объектами. 

samp_dist = calc_mahalanobis_square_dist(means, data[FEATURES], cov)
samp_dist.head()

NameError: name 'means' is not defined

In [42]:
# Зададим функцию для вывода апостериорных вероятностей.

def LDA_predict_proba(lda, X):
    return pd.DataFrame(
        lda.predict_proba(X),
        columns=lda.classes_,
        index=X.index
    )

In [43]:
lda_post_prob = LDA_predict_proba(lda, data[FEATURES])
lda_post_prob.head()

NameError: name 'lda' is not defined

In [44]:
# Зададим функцию расчета Лямбды Уилкса:

def wilks_lambda(samples, labels):
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    dT = np.linalg.det(scatter_matrix(samples)) # расчет определителя матрицы рассеяния
    dE = np.linalg.det(classes_scatter_matrix(samples, labels)) # расчет определителя матрицы классового рас-сеяния
    return dE / dT # их частное и есть Лямбда Уилкса

In [45]:
#Зададим функцию расчета f value и p value :

#•	lmbda - Лямбда Уилкса
#•	n_obs - кол-во объектов
#•	n_var - кол-во признаков
#•	n_cls - кол-во классов

def f_value_wilks_lambda(lmbda, n_obs, n_var, n_cls):
    num = (1 - lmbda) * (n_obs - n_cls - n_var)
    den = lmbda * (n_cls - 1)
    f_value = num / den #значение статистики Фишера
    # f.sf считает p value для определенного значения статистики Фишера и заданных степеней свободы
    p_value = f.sf(f_value, n_cls - 1, n_obs - n_cls - n_var) 
    return f_value, p_value

In [46]:
#sda - функция для проведения пошагового дискриминантного анализа:
#•	samples - набор объектов
#•	labels - названия классов
#•	F_in=1e-4 - установленное нами значение F_to_Enter Функция выводит для каждого шага переменные 
#в модели(into) и вне модели(out) и их характеристики.
def sda(samples, labels, F_in=1e-4):
    stat_columns = ["Wilk's lambda", "Partial lamb-da", "F to enter", "p-value"]
    n_cls = labels.unique().size # количество классов
    n_obs = samples.shape[0] # количество объектов
    
    # словари для вывода переменных в модели и вне модели на каждом шаге
    out = {0: pd.DataFrame(columns=stat_columns, index=samples.columns, dtype=float)} 
    into = {0: pd.DataFrame(columns=stat_columns, dtype=float)}
    
    step = 0
    while True:
        model_lambda = wilks_lambda(samples[into[step].index], labels) #по-считали Лямбду Уилкса для модели на данном шаге
        for x in out[step].index: # проходим по пере-менным вне модели на данном шаге
        # далее рассчитываем характеристики для дан-ных переменных и записываем их в таблицу    
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [x]], labels) 
            partial_lambda = lmbda / model_lambda
            f_lambda, p_value = f_value_wilks_lambda(partial_lambda, n_obs, into[step].index.size, n_cls)
            out[step].loc[x] = lmbda, partial_lambda, f_lambda, p_value 
            
        for x in into[step].index: # проходим по пе-ременным в модели на данном шаге
        # далее рассчитываем характеристики для дан-ных переменных и записываем их в таблицу  
            lmbda = wilks_lambda(samples[into[step].index.drop(x)], labels)
            partial_lambda = model_lambda / lmbda
            f_lambda, p_value = f_value_wilks_lambda(partial_lambda, n_obs, into[step].index.size - 1, n_cls)
            into[step].loc[x] = lmbda, partial_lambda, f_lambda, p_value
        # критерий для остановки цикла 
        # если вне модели нет переменных ИЛИ новая переменная обладает f_to_enter меньше порогового зна-чения, цикл остановлен
        if out[step].index.size == 0 or out[step]["F to enter"].max() < F_in:
            break
        # ищем элемент с max f_to_enter
        x_to_enter = out[step]["F to enter"].idxmax()
        # переносим его из элементов "вне модели" в элементы "в модели"
        into[step + 1] = into[step].append(out[step].loc[x_to_enter])
        out[step + 1] = out[step].drop(index=x_to_enter)
        
        step += 1 # переходим к следующему шагу
    return into, out


In [47]:
into, out = sda(train_data[FEATURES], train_data.Class, 2.2)

NameError: name 'train_data' is not defined

In [48]:
for i, t in into.items():
    print("Step ", i)
    print(t, end="\n\n")

NameError: name 'into' is not defined

In [49]:
# Выводим результаты работы для переменных "вне модели" на экран.

for i, t in out.items():
    print("Step ", i)
    print(t, end="\n\n")

NameError: name 'out' is not defined

In [50]:
# Выводим названия признаков в модели.

stepwise_features = into[len(into) - 1].index.tolist()
stepwise_features

NameError: name 'into' is not defined

In [51]:
# Проводим дискриминантный анализ для отобранных признаков.

stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[stepwise_features], train_data.Class)



NameError: name 'train_data' is not defined

In [52]:
# Выводим коэффициенты функции Фишера на экран.

stepwise_df_coef = get_df_coef(stepwise_lda, stepwise_features)
stepwise_df_coef

NameError: name 'stepwise_lda' is not defined

In [53]:
# Получим полную картину классификации всех объектов. 

stepwise_lda_predict = LDA_predict(stepwise_lda, data[stepwise_features])
stepwise_lda_predict.head()

NameError: name 'stepwise_lda' is not defined

In [54]:
data_to_excel["Results stepwise lda"] = stepwise_lda_predict
data_to_excel.head()

NameError: name 'stepwise_lda_predict' is not defined

In [55]:
data_to_excel.to_excel("RESULT.xlsx")

NameError: name 'data_to_excel' is not defined

In [56]:
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline
data_to_plot = pd.read_excel("RESULT.xlsx")
df = data_to_plot[['X1', 'X5', 'Results stepwise lda']]
categories = np.unique(df['Results stepwise lda'])
colors = [plt.cm.tab10(i/float(len(categories)-1)) for i in range(len(categories))]
plt.figure(figsize=(16, 10), dpi= 90, facecolor='w', edgecolor='k')
for i, category in enumerate(categories):
    plt.scatter('X1', 'X5', 
                data=df.loc[df['Results stepwise lda']==category, :], 
                s=20, c=colors[i], label=str(category))

# Decorations
plt.gca().set(xlim=(0.0, 21.0), ylim=(0, 3.0),
              xlabel='Производительность труда', ylabel='Фондоотдача активной части основных производственных фон-дов')

plt.xticks(fontsize=12); plt.yticks(fontsize=12)
plt.title("Пошаговый дискриминантный анализ", fontsize=22)
plt.legend(fontsize=12)    
plt.show()  

OSError: 'seaborn-whitegrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)